In [2]:
import os
os.chdir("/home/atulgang/Thesis")


In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-06 15:39:57,644 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-04-29 17:42:08,805 : INFO : loading Dictionary object from dict/lem10
2018-04-29 17:42:08,855 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-04-29 17:42:08,952 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-04-29 17:42:08,953 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-04-29 17:42:08,953 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
%%time
# Tfidf model
# Will take time as to generate a model it processes over the whole corpus
tfidf = gensim.models.TfidfModel(corpus)
# save tfidf model
tfidf.save('py_models/tfidf_lem10.mdl')

2018-04-29 17:42:09,035 : INFO : collecting document frequencies
2018-04-29 17:42:09,035 : INFO : PROGRESS: processing document #0
2018-04-29 17:42:10,803 : INFO : PROGRESS: processing document #10000
2018-04-29 17:42:12,690 : INFO : PROGRESS: processing document #20000
2018-04-29 17:42:14,521 : INFO : PROGRESS: processing document #30000
2018-04-29 17:42:16,378 : INFO : PROGRESS: processing document #40000
2018-04-29 17:42:17,842 : INFO : PROGRESS: processing document #50000
2018-04-29 17:42:19,561 : INFO : PROGRESS: processing document #60000
2018-04-29 17:42:21,143 : INFO : PROGRESS: processing document #70000
2018-04-29 17:42:22,974 : INFO : PROGRESS: processing document #80000
2018-04-29 17:42:24,919 : INFO : PROGRESS: processing document #90000
2018-04-29 17:42:27,130 : INFO : PROGRESS: processing document #100000
2018-04-29 17:42:29,328 : INFO : PROGRESS: processing document #110000
2018-04-29 17:42:30,553 : INFO : PROGRESS: processing document #120000
2018-04-29 17:42:32,367 : 

CPU times: user 56 s, sys: 424 ms, total: 56.4 s
Wall time: 56.4 s


In [10]:
##################################### load tfidf model
tfidf = gensim.models.tfidfmodel.TfidfModel.load('py_models/tfidf_lem10.mdl')

2018-04-29 17:43:05,410 : INFO : loading TfidfModel object from py_models/tfidf_lem10.mdl
2018-04-29 17:43:05,501 : INFO : loaded py_models/tfidf_lem10.mdl


In [11]:
%%time
# tfidf wrapper coprus
# doesn't take time - yield
tfidf_corpus = tfidf[corpus]

# actual tfidf corpus serialized
corpora.MmCorpus.serialize('corpus/tfidf_lem10.mm', tfidf_corpus)


2018-04-29 17:43:05,635 : INFO : storing corpus in Matrix Market format to corpus/tfidf_lem10.mm
2018-04-29 17:43:05,636 : INFO : saving sparse matrix to corpus/tfidf_lem10.mm
2018-04-29 17:43:05,638 : INFO : PROGRESS: saving document #0
2018-04-29 17:43:07,478 : INFO : PROGRESS: saving document #1000
2018-04-29 17:43:08,806 : INFO : PROGRESS: saving document #2000
2018-04-29 17:43:10,353 : INFO : PROGRESS: saving document #3000
2018-04-29 17:43:12,209 : INFO : PROGRESS: saving document #4000
2018-04-29 17:43:14,512 : INFO : PROGRESS: saving document #5000
2018-04-29 17:43:16,109 : INFO : PROGRESS: saving document #6000
2018-04-29 17:43:17,784 : INFO : PROGRESS: saving document #7000
2018-04-29 17:43:20,392 : INFO : PROGRESS: saving document #8000
2018-04-29 17:43:22,222 : INFO : PROGRESS: saving document #9000
2018-04-29 17:43:24,589 : INFO : PROGRESS: saving document #10000
2018-04-29 17:43:26,605 : INFO : PROGRESS: saving document #11000
2018-04-29 17:43:28,452 : INFO : PROGRESS: sa

2018-04-29 17:46:51,044 : INFO : PROGRESS: saving document #122000
2018-04-29 17:46:53,084 : INFO : PROGRESS: saving document #123000
2018-04-29 17:46:55,276 : INFO : PROGRESS: saving document #124000
2018-04-29 17:46:57,913 : INFO : PROGRESS: saving document #125000
2018-04-29 17:47:00,236 : INFO : PROGRESS: saving document #126000
2018-04-29 17:47:02,136 : INFO : PROGRESS: saving document #127000
2018-04-29 17:47:03,663 : INFO : PROGRESS: saving document #128000
2018-04-29 17:47:05,237 : INFO : PROGRESS: saving document #129000
2018-04-29 17:47:06,445 : INFO : PROGRESS: saving document #130000
2018-04-29 17:47:07,560 : INFO : PROGRESS: saving document #131000
2018-04-29 17:47:08,696 : INFO : PROGRESS: saving document #132000
2018-04-29 17:47:09,816 : INFO : PROGRESS: saving document #133000
2018-04-29 17:47:11,106 : INFO : PROGRESS: saving document #134000
2018-04-29 17:47:12,167 : INFO : PROGRESS: saving document #135000
2018-04-29 17:47:13,224 : INFO : PROGRESS: saving document #13

2018-04-29 17:50:06,732 : INFO : PROGRESS: saving document #245000
2018-04-29 17:50:07,299 : INFO : PROGRESS: saving document #246000
2018-04-29 17:50:07,982 : INFO : PROGRESS: saving document #247000
2018-04-29 17:50:08,613 : INFO : PROGRESS: saving document #248000
2018-04-29 17:50:09,242 : INFO : PROGRESS: saving document #249000
2018-04-29 17:50:09,958 : INFO : PROGRESS: saving document #250000
2018-04-29 17:50:10,650 : INFO : PROGRESS: saving document #251000
2018-04-29 17:50:11,287 : INFO : PROGRESS: saving document #252000
2018-04-29 17:50:11,870 : INFO : PROGRESS: saving document #253000
2018-04-29 17:50:12,399 : INFO : PROGRESS: saving document #254000
2018-04-29 17:50:12,912 : INFO : PROGRESS: saving document #255000
2018-04-29 17:50:13,375 : INFO : PROGRESS: saving document #256000
2018-04-29 17:50:13,831 : INFO : PROGRESS: saving document #257000
2018-04-29 17:50:14,277 : INFO : PROGRESS: saving document #258000
2018-04-29 17:50:14,731 : INFO : PROGRESS: saving document #25

CPU times: user 9min 21s, sys: 3.45 s, total: 9min 24s
Wall time: 9min 26s


In [12]:
##################################### load tfidf corpus
tfidf_corpus = corpora.MmCorpus('corpus/tfidf_lem10.mm')

2018-04-29 17:52:32,381 : INFO : loaded corpus index from corpus/tfidf_lem10.mm.index
2018-04-29 17:52:32,382 : INFO : initializing cython corpus reader from corpus/tfidf_lem10.mm
2018-04-29 17:52:32,382 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [13]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [14]:
%%time
# create index - Tfidf
tfidf_index = gensim.similarities.Similarity('py_models/tfidf_lem10.index',tfidf_corpus,len(dictionary))

# save index - tfidf
gensim.similarities.Similarity.save(tfidf_index,'py_models/TFIDF_lem10_index')


2018-04-29 17:52:32,562 : INFO : starting similarity index under py_models/tfidf_lem10.index
2018-04-29 17:52:37,110 : INFO : PROGRESS: fresh_shard size=10000
2018-04-29 17:52:42,448 : INFO : PROGRESS: fresh_shard size=20000
2018-04-29 17:52:47,068 : INFO : PROGRESS: fresh_shard size=30000
2018-04-29 17:52:48,356 : INFO : creating sparse index
2018-04-29 17:52:48,356 : INFO : creating sparse matrix from corpus
2018-04-29 17:52:48,357 : INFO : PROGRESS: at document #0/32768
2018-04-29 17:52:53,527 : INFO : PROGRESS: at document #10000/32768
2018-04-29 17:52:59,000 : INFO : PROGRESS: at document #20000/32768
2018-04-29 17:53:04,175 : INFO : PROGRESS: at document #30000/32768
2018-04-29 17:53:05,426 : INFO : created <32768x169716 sparse matrix of type '<type 'numpy.float32'>'
	with 10888040 stored elements in Compressed Sparse Row format>
2018-04-29 17:53:05,427 : INFO : creating sparse shard #0
2018-04-29 17:53:05,428 : INFO : saving index shard to py_models/tfidf_lem10.index.0
2018-04-2

2018-04-29 17:55:42,667 : INFO : saved py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,667 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,668 : INFO : loading index from py_models/tfidf_lem10.index.5.index.npy with mmap=r
2018-04-29 17:55:42,669 : INFO : loaded py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,703 : INFO : PROGRESS: fresh_shard size=0
2018-04-29 17:55:47,822 : INFO : PROGRESS: fresh_shard size=10000
2018-04-29 17:55:52,892 : INFO : PROGRESS: fresh_shard size=20000
2018-04-29 17:55:56,233 : INFO : PROGRESS: fresh_shard size=30000
2018-04-29 17:55:56,985 : INFO : creating sparse index
2018-04-29 17:55:56,986 : INFO : creating sparse matrix from corpus
2018-04-29 17:55:56,987 : INFO : PROGRESS: at document #0/32768
2018-04-29 17:56:03,380 : INFO : PROGRESS: at document #10000/32768
2018-04-29 17:56:09,667 : INFO : PROGRESS: at document #20000/32768
2018-04-29 17:56:12,513 : INFO : PROGRESS: at document #30000/32768


CPU times: user 4min 58s, sys: 2.48 s, total: 5min
Wall time: 5min 8s


In [15]:
########################### load tfidf index
tfidf_index = gensim.similarities.Similarity.load('py_models/TFIDF_lem10_index')

2018-04-29 17:57:41,123 : INFO : loading Similarity object from py_models/TFIDF_lem10_index
2018-04-29 17:57:41,123 : INFO : loaded py_models/TFIDF_lem10_index


In [16]:
%%time

# calculate similarity for all 
tfidf_index.num_best= None
i#######################         SIMILIRATIES SCORES AND RANKING    ###########################
results = copy.deepcopy(query_truth)
### lists of list (indexing from 1, dummy element)
sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    # inside the square bracket determines query representation
    
    sims = tfidf_index[tfidf[dictionary.doc2bow(query)]]
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        #x.append(common_words(filenames.index(x[0]+'.txt'), tfidf_corpus, query))
    
    # sort wrt relevance(from truth) and then ranks(from our model)
    #tfidf_results[str(i+1)].sort(key=lambda x: (-int(x[1]),x[3]))     
    
        

2018-04-29 17:57:43,118 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.0
2018-04-29 17:57:43,119 : INFO : loading index from py_models/tfidf_lem10.index.0.index.npy with mmap=r
2018-04-29 17:57:43,120 : INFO : loaded py_models/tfidf_lem10.index.0
2018-04-29 17:57:43,157 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.1
2018-04-29 17:57:43,203 : INFO : loaded py_models/tfidf_lem10.index.1
2018-04-29 17:57:43,237 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.2
2018-04-29 17:57:43,238 : INFO : loading index from py_models/tfidf_lem10.index.2.index.npy with mmap=r
2018-04-29 17:57:43,240 : INFO : loaded py_models/tfidf_lem10.index.2
2018-04-29 17:57:43,282 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.3
2018-04-29 17:57:43,338 : INFO : loaded py_models/tfidf_lem10.index.3
2018-04-29 17:57:43,372 : INFO : loading SparseMatrixSimilarity object from py_models/tf

CPU times: user 6.72 s, sys: 240 ms, total: 6.96 s
Wall time: 7.13 s


In [17]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.16185194, 205],
  ['ConsumerCourt_DCDRC_106530', '0', 0.066299856, 2049],
  ['ConsumerCourt_DCDRC_107608', '1', 0.08884313, 1231],
  ['ConsumerCourt_DCDRC_114291', '1', 0.12928788, 532],
  ['ConsumerCourt_DCDRC_114382', '1', 0.10607624, 893],
  ['ConsumerCourt_DCDRC_118185', '1', 0.09642149, 1088],
  ['ConsumerCourt_DCDRC_130318', '0', 0.1199432, 653],
  ['ConsumerCourt_DCDRC_130570', '1', 0.27709824, 7],
  ['ConsumerCourt_DCDRC_131146', '1', 0.06343228, 2197],
  ['ConsumerCourt_DCDRC_131717', '1', 0.08180869, 1438],
  ['ConsumerCourt_DCDRC_131741', '1', 0.07304419, 1746],
  ['ConsumerCourt_DCDRC_131818', '1', 0.09741112, 1068],
  ['ConsumerCourt_DCDRC_131950', '0', 0.05442484, 2824],
  ['ConsumerCourt_DCDRC_131972', '0', 0.08816901, 1251],
  ['ConsumerCourt_DCDRC_132932', '0', 0.06848099, 1925],
  ['ConsumerCourt_DCDRC_133592', '1', 0.1699525, 155],
  ['ConsumerCourt_DCDRC_134386', '0', 0.33104795, 1],
  ['ConsumerCourt_DCDRC_135474', '1', 

In [18]:
model_name = 'TFIDF_lem10'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [19]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [6]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

model_name = 'TFIDF_lem10'
dest_filename = 'result_excel/' + model_name + '.xlsx'

f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)